In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import json
from tqdm import tqdm
import pandas as pd

import os
import sys
sys.path.append('..')
from utils import nethook
from utils import model_utils

In [3]:
MODEL_NAME = "gpt2-medium"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=False)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [4]:
prompt = [
    "This was an awesome movie",
    "This was a bad movie",
    "This was not a good movie",
    "That movie was garbage",
    "Iphone 7 is not a good phone",
    "Google new line of pixels are great"
]

prompt = ["<REVIEW>: " + p + " <SENTIMENT>" for p in prompt]

txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    # debug=True,
    get_answer_tokens=True,

    # track_interesting_words = [
    #     [" positive", " negative"],
    #     [" positive", " negative"]
    # ]
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> <REVIEW>: I love this movie <REVIEW>: I love this movie <REVIEW>: I love this movie <
p(answer):  p(' <'[1279])=0.0664, p('
'[198])=0.0569, p(' :'[1058])=0.05, p(' I'[314])=0.0458, p(' It'[632])=0.0208

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> <REVIEW>: I don't know what to say <REVIEW>: I'm not sure what to say <REVIEW
p(answer):  p(' <'[1279])=0.0586, p(' :'[1058])=0.0528, p('
'[198])=0.0445, p(' I'[314])=0.0341, p(' ('[357])=0.0278

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This was not a good movie <SENTIMENT> <REVIEW>: I don't think it was a good movie <REVIEW>: I think it was a good movie
p(answer):  p(' <'[1279])=0.0508, p(' :'[1058])=0.0455, p(' I'[314])=0.0397, p('
'[198])=0.0332, p(' ('[357])=0.0229

<REVIEW>: That movie was garbage <SENTIMENT>
<REVIEW>: That movie was garbage <SENTIMENT> <REVIEW>: I'm not going to watch 

## Prompt Tuning

In [5]:
soft_prompts = torch.load("Saved_weights/Final/Prompt_Tuning/gpt2-medium/prompt_size__8.pth", map_location='cuda:0')
txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    get_answer_tokens=True,
    light_weight_tuning= soft_prompts, algo = "prompt"
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><REVIEW>: This was an awesome movie <SENTIMENT> positive review positive review positive review positive review positive review positive review positive review positive review positive
p(answer):  p(' positive'[3967])=0.9604, p(' negative'[4633])=0.0322, p(' I'[314])=0.0009, p(' positives'[38548])=0.0008, p(' good'[922])=0.0006

<REVIEW>: This was a bad movie <SENTIMENT>
<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><REVIEW>: This was a bad movie <SENTIMENT> negative review negative review negative review negative review negative review negative review negative review negative review negative
p(answer):  p(' negative'[4633])=0.9807, p(' positive'[3967])=0.0155, p(' bad'[2089])=0.0004, p(' poor'[3595])=0.0004, p(' I'[314])=0.0004

<REVIEW>: This was not a good movie <SENTIMENT>
<|

## Prefix-tuning

In [6]:
prefix_embeddings = torch.load("Saved_weights/Final/Prefix_Tuning/gpt2-medium/prefix_size__2.pth")

In [7]:
txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    get_answer_tokens=True,
    light_weight_tuning= prefix_embeddings, algo = "prefix"
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<|endoftext|><|endoftext|><REVIEW>: This was an awesome movie <SENTIMENT> positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' positive'[3967])=0.9777, p(' negative'[4633])=0.0212, p(' great'[1049])=0.0001, p(' good'[922])=0.0001, p('positive'[24561])=0.0

<REVIEW>: This was a bad movie <SENTIMENT>
<|endoftext|><|endoftext|><REVIEW>: This was a bad movie <SENTIMENT> negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative negative
p(answer):  p(' negative'[4633])=0.9861, p(' positive'[3967])=0.0133, p(' bad'[2089])=0.0001, p(' negatives'[42510])=0.0001, p(' Negative'[36183])=0.0

<REVIEW>: This was not a good movie <SENTIMENT>
<|endo

## Adapter Tuning

In [8]:
from torch.nn import Linear, ReLU, Sequential, Tanh

def init_weights(m, lo = -0.0001, hi = 0.0001):
    if isinstance(m, Linear):
        torch.nn.init.uniform_(m.weight, a = lo, b = hi)
        torch.nn.init.uniform_(m.bias, a = lo, b = hi)


class Adapter(torch.nn.Module):
    def __init__(self, inp_out_dim, adapter_dim, hidden_conf = []):
        super().__init__()
        self.inp_out_dim = inp_out_dim
        self.conf = [inp_out_dim] + hidden_conf + [adapter_dim] + hidden_conf[::-1] + [inp_out_dim]
        # print(self.conf)
        self.adapter_dim = adapter_dim
        self.layers = []

        i = 1
        while i < len(self.conf):
            inp = self.conf[i-1]
            out = self.conf[i]
            layer_name = f'layer{i}'
            setattr(self, layer_name, Sequential(Linear(inp, out), ReLU()))
            self.layers.append(layer_name)
            i += 1
    
    def forward(self, x):
        x_init = x.clone()
        for module in self.named_children():
            layer_name = module[0]
            layer = getattr(self, layer_name)
            x = layer(x)
        return x + x_init

In [9]:
adapter_blocks = torch.load("Saved_weights/Final/Adapters/gpt2-medium/adapter_dim__32.pth")

In [10]:
txt, ret_dict = model_utils.generate_fast(
    model, tokenizer,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    get_answer_tokens=True,
    light_weight_tuning= adapter_blocks, algo = "adapter"
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> positive. I was blown away by the way the cast and crew handled the situation. I was also blown away by the way
p(answer):  p(' positive'[3967])=0.7549, p(' awesome'[7427])=0.114, p(' great'[1049])=0.0158, p(' amazing'[4998])=0.0147, p(' good'[922])=0.0062

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> negative. I don't know what the hell I was thinking. I was so disappointed. I was so disappointed that I was
p(answer):  p(' negative'[4633])=0.7905, p(' bad'[2089])=0.0838, p(' positive'[3967])=0.0202, p(' terrible'[7818])=0.0091, p(' BAD'[33934])=0.0067

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This was not a good movie <SENTIMENT> negative. I was not impressed with the story, the acting, the direction, the acting, the direction, the acting
p(answer):  p(' negative'[4633])=0.8288, p(' positive'[3967])=0.0465, p(' bad'[2089])=0.0125, p(

## Fine-tuning

In [11]:
mt_2 = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=False)
model_2 = mt_2.model
tokenizer_2 = mt_2.tokenizer
tokenizer_2.pad_token = tokenizer_2.eos_token
print(f"Duplicate Model {MODEL_NAME} initialized")

Duplicate Model gpt2-medium initialized


In [12]:
checkpoint = torch.load("Saved_weights/Final/Finetune/gpt2-medium/gpt2-medium____data_40000.pth")
print("loaded checkpoint")
model_2.load_state_dict(checkpoint)
print("loaded model weights")

loaded checkpoint
loaded model weights


In [13]:
txt, ret_dict = model_utils.generate_fast(
    model_2, tokenizer_2,
    prompt,
    argmax_greedy = True,
    max_out_len= 40,
    # debug=True,
    get_answer_tokens=True,
)

model_utils.print_formatted_results(prompt, txt, ret_dict)

<REVIEW>: This was an awesome movie <SENTIMENT>
<REVIEW>: This was an awesome movie <SENTIMENT> positive negative negative negative negative negative negative negative negative negative positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' positive'[3967])=0.9651, p(' negative'[4633])=0.0349, p(' Positive'[33733])=0.0, p(' positives'[38548])=0.0, p('positive'[24561])=0.0

<REVIEW>: This was a bad movie <SENTIMENT>
<REVIEW>: This was a bad movie <SENTIMENT> negative positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive
p(answer):  p(' negative'[4633])=0.849, p(' positive'[3967])=0.151, p(' Negative'[36183])=0.0, p(' Positive'[33733])=0.0, p(' positives'[38548])=0.0

<REVIEW>: This was not a good movie <SENTIMENT>
<REVIEW>: This was 